In [1]:
%load_ext autoreload
import os
import sys
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
sys.path.append(os.path.join('../src/utils'))
sys.path.append(os.path.join('../src/'))
import helpers
import eda_functions
import streamlit_funcs

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
DB1_FOLDER = '../data/db1'
DB2_FOLDER = '../data/db2'

In [4]:
db1 = streamlit_funcs.load_db1(DB1_FOLDER)

In [5]:
ext1, ext2 = streamlit_funcs.load_db2(DB2_FOLDER)

# DB1

In [7]:
db1.columns = [col if col != 'catch_date' else 'date' for col in db1.columns]
db1['date'] = pd.to_datetime(db1['date']).dt.date
db1_aggregated = db1.groupby(['id_ves', 'date', 'id_fish'])['catch_volume'].sum().reset_index()
display(db1_aggregated.head())

,id_ves,date,id_fish,catch_volume
0,2,2022-01-10,848,17.0
1,2,2022-01-11,848,20.0
2,2,2022-01-22,849,1342.0
3,2,2022-01-23,849,2078.0
4,2,2022-01-24,849,97.0


# DB2

In [8]:
ext1.columns = [col if col != 'date_fishery' else 'date' for col in ext1.columns]
ext1['date'] = pd.to_datetime(ext1['date']).dt.date
db2_merged = ext2.merge(ext1, left_on=['id_vsd'], right_on=['id_vsd'], suffixes=['_ext', '_ext2'], how='left')

In [9]:
db2_aggregated = db2_merged.groupby(['id_ves', 'date', 'id_fish'])['volume'].sum().reset_index()
db2_aggregated['id_ves'] = db2_aggregated['id_ves'].astype(int)
db2_aggregated['id_fish'] = db2_aggregated['id_fish'].astype(int)

In [10]:
joined_bases = db1_aggregated.merge(db2_aggregated, on=['id_ves', 'id_fish', 'date'], how='inner')

joined_bases['volume_div_1000'] = joined_bases['volume'] / 1000
joined_bases['volume_div_100'] = joined_bases['volume'] / 100
joined_bases['delta1'] = abs(joined_bases['catch_volume'] - joined_bases['volume_div_1000']) / joined_bases['catch_volume']
joined_bases['delta2'] = abs(joined_bases['catch_volume'] - joined_bases['volume_div_100']) / joined_bases['catch_volume']
joined_bases['delta3'] = abs(joined_bases['catch_volume'] - joined_bases['volume']) / joined_bases['catch_volume']

joined_bases['mismatch, %'] = joined_bases.apply(lambda x: min(x['delta1'], x['delta2'], x['delta3']), axis=1)
joined_bases['mismatch, %'] = (100 * joined_bases['mismatch, %']).round(2)

joined_bases = joined_bases.drop(columns=['delta1', 'delta2', 'delta3'])

threshold = 25
joined_bases['threshold_volume'] = threshold
joined_bases['is_abnormal'] = joined_bases['mismatch, %'] > joined_bases['threshold_volume']

In [11]:
joined_bases[joined_bases['is_abnormal']]

,id_ves,date,id_fish,catch_volume,volume,volume_div_1000,volume_div_100,"mismatch, %",threshold_volume,is_abnormal
618,1288,2022-03-10,292,5118.00,3164,3.164,31.64,38.18,25,True
739,1289,2022-03-27,88,2159.92,5691,5.691,56.91,97.37,25,True
740,1289,2022-03-27,147,292.50,929,0.929,9.29,96.82,25,True
741,1289,2022-03-27,292,30511.50,63505,63.505,635.05,97.92,25,True
743,1289,2022-03-27,410,10552.95,22291,22.291,222.91,97.89,25,True
...,...,...,...,...,...,...,...,...,...,...
44918,1654,2022-01-02,294,341.00,655,0.655,6.55,92.08,25,True
45297,1656,2022-02-20,400,238753.00,441763,441.763,4417.63,85.03,25,True
45343,1656,2022-04-09,400,309309.00,618690,618.690,6186.90,98.00,25,True
45477,1659,2022-02-20,400,24100.00,48200,48.200,482.00,98.00,25,True
